In [1]:
import pandas as pd
from siuba import *

In [2]:
scolist = pd.read_excel('gs://calitp-analytics-data/data-analyses/ntd/RTPAs Contact Information FY 2021-22.xlsx')

In [3]:
scolist.head()

,ENTITY_NAME,FISCAL_YEAR,CHAIR_FIRST_NAME,CHAIR_MI,CHAIR_LAST_NAME,CHAIR_PHONE,CHAIR_EMAIL,EXEC_FIRST_NAME,EXEC_MI,EXEC_LAST_NAME,...,FISCAL_OFFICER_MI,FISCAL_OFFICER_LAST_NAME,FISCAL_OFFICER_PHONE,FISCAL_OFFICER_EMAIL,STREET_ADDRESS1,STREET_ADDRESS2,CITY,STATE,ZIP,ADDRESS_CHANGE
0,Alpine County Transportation Commission,2022,David,NaN,Griffith,(530) 694-2168,dgriffith.9@gmail.com,JT,NaN,Chevalier,...,NaN,Leitenbauer,(530) 694-2284 ext.134,kleitenbauer@alpinecountyca.gov,Alpine County Community Development,50 Diamond Valley Road,Markleeville,CA,96120,False
1,Amador County Transportation Commission,2022,Patrick,NaN,Crew,(209) 267-2282,pcrew@amadorgov.org,John,NaN,Gedney,...,NaN,Yakesh,(209) 267-2282,terri@actc-amador.org,117 Valley View Way,NaN,Sutter Creek,CA,95685,False
2,Butte County Association of Governments,2022,Jodie,NaN,Jones,(530) 538-6834,jjones@buttecounty.net,Jon,A,Clark,...,P,Quinn,(530) 809-4616,jquinn@bcag.org,"326 Huss Drive, Suite 150",NaN,Chico,CA,95928-8441,False
3,Calaveras Council of Governments,2022,Amanda,NaN,Folendorf,(209) 754-2094,NaN,Michael,NaN,Azevedo,...,NaN,Navarro,(530) 458-0400,agnavarro@countyofcolusa.org,"546 Jay Street, Suite 202",NaN,Colusa,CA,95932-2489,False
4,Colusa County Local Transportation Commission,2022,Frank,NaN,Kennedy,(530) 473-2955 ext.117,fkennedy@cityofwilliams.org,Mary,NaN,Gilbert,...,NaN,Rivera,(831) 637-7665,norma@sanbenitocog.org,"330 Tres Pinos Road, Suite C7",NaN,Hollister,CA,95023,False


In [4]:
from siuba.experimental.pivot import pivot_wider, pivot_longer
from siuba import Fx

In [18]:
fns = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_FIRST_NAME")) 
        >> pivot_longer(_.contains("_FIRST_NAME"), names_to="position", values_to="first_name")
        >> mutate(position=_.position.str.replace('_FIRST_NAME', '', regex=False))
      )

In [22]:
lns = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_LAST_NAME")) 
        >> pivot_longer(_.contains("_LAST_NAME"), names_to="position", values_to="last_name") 
        >> mutate(position=_.position.str.replace('_LAST_NAME', '', regex=False))
      )

In [24]:
emails = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_EMAIL")) 
        >> pivot_longer(_.contains("_EMAIL"), names_to="position", values_to="email") 
        >> mutate(position=_.position.str.replace('_EMAIL', '', regex=False))
      )

In [25]:
contact_names = fns.merge(lns).reset_index(drop=True)

In [27]:
contacts = contact_names.merge(emails).reset_index(drop=True)

In [28]:
contacts.head()

,ENTITY_NAME,position,first_name,last_name,email
0,Alpine County Transportation Commission,CHAIR,David,Griffith,dgriffith.9@gmail.com
1,Alpine County Transportation Commission,EXEC,JT,Chevalier,jtchevallier@alpinecountyca.gov
2,Alpine County Transportation Commission,FISCAL_OFFICER,Klaus,Leitenbauer,kleitenbauer@alpinecountyca.gov
3,Amador County Transportation Commission,CHAIR,Patrick,Crew,pcrew@amadorgov.org
4,Amador County Transportation Commission,EXEC,John,Gedney,john@actc-amador.org


In [29]:
contacts.to_excel("gs://calitp-analytics-data/data-analyses/ntd/sco_rtpa_contacts_long.xlsx", index=False)